# Análisis Univariante: Autoeficacia

## Estudio exhaustivo de autoeficacia de estudiantes

Este notebook realiza un análisis univariante detallado de la autoeficacia de estudiantes en la UPV, incluyendo:
- Estadísticas descriptivas
- Análisis de distribución
- Visualizaciones profesionales
- Detección de outliers
- Evaluación de calidad de datos

## 1. Librerías Requeridas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import shapiro, kstest, norm, skew, kurtosis
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('Librerías cargadas correctamente')

## 2. Cargar y Explorar Datos de Autoeficacia

In [ ]:
# Cargar el panel maestro
panel_maestro = pd.read_csv('../../data_extraction/panel_maestro_UPV.csv', encoding='utf-8')

# Información general
print("Dimensión del dataset:", panel_maestro.shape)
print("\nColumnas disponibles:")
print(panel_maestro.columns.tolist())

# Seleccionar columnas de autoeficacia
autoeficacia_cols = ['autoeficacia_3_anos', 'nivel_autoeficacia']

print("\n" + "="*80)
print("INFORMACIÓN DE VARIABLES DE AUTOEFICACIA")
print("="*80)

for col in autoeficacia_cols:
    if col in panel_maestro.columns:
        print(f"\n{col}:")
        print(f"  Tipo de dato: {panel_maestro[col].dtype}")
        print(f"  Námero de observaciones: {len(panel_maestro[col])}")
        print(f"  Válidos: {panel_maestro[col].notna().sum()}")
        print(f"  Ná Faltantes: {panel_maestro[col].isna().sum()}")
        print(f"  Porcentaje válidos: {(panel_maestro[col].notna().sum() / len(panel_maestro[col]) * 100):.2f}%")

# Primeras filas
print("\n" + "="*80)
print("PRIMERAS 10 FILAS")
print("="*80)
print(panel_maestro[autoeficacia_cols].head(10))

## 3. Estadísticas Descriptivas de Autoeficacia

In [ ]:
# Trabajar con datos válidos
autoeficacia_data = panel_maestro['autoeficacia_3_anos'].dropna()
nivel_autoeficacia_data = panel_maestro['nivel_autoeficacia'].dropna()

print("\n" + "="*80)
print("ESTÁDÍSTICAS DESCRIPTIVAS - AUTOEFICACIA (0-10)")
print("="*80)

print(f"\nNúmero de observaciones válidas: {len(autoeficacia_data)}")
print(f"\nMedidas de Tendencia Central:")
print(f"  Media:                    {autoeficacia_data.mean():.4f}")
print(f"  Mediana:                  {autoeficacia_data.median():.4f}")
print(f"  Moda:                     {autoeficacia_data.mode().values[0]:.4f}")

print(f"\nMedidas de Dispersión:")
print(f"  Desviación Estándar:     {autoeficacia_data.std():.4f}")
print(f"  Varianza:                 {autoeficacia_data.var():.4f}")
print(f"  Rango:                    {autoeficacia_data.max() - autoeficacia_data.min():.4f}")
print(f"  Mínimo:                  {autoeficacia_data.min():.4f}")
print(f"  Máximo:                  {autoeficacia_data.max():.4f}")

print(f"\nCuartiles e IQR:")
q1 = autoeficacia_data.quantile(0.25)
q3 = autoeficacia_data.quantile(0.75)
iqr = q3 - q1
print(f"  Q1 (25%):                 {q1:.4f}")
print(f"  Q2 (50%):                 {autoeficacia_data.quantile(0.50):.4f}")
print(f"  Q3 (75%):                 {q3:.4f}")
print(f"  IQR (Q3-Q1):              {iqr:.4f}")

print(f"\nPercentiles Adicionales:")
print(f"  P10:                      {autoeficacia_data.quantile(0.10):.4f}")
print(f"  P25:                      {autoeficacia_data.quantile(0.25):.4f}")
print(f"  P50:                      {autoeficacia_data.quantile(0.50):.4f}")
print(f"  P75:                      {autoeficacia_data.quantile(0.75):.4f}")
print(f"  P90:                      {autoeficacia_data.quantile(0.90):.4f}")

print("\n" + "="*80)
print("DISTRIBUCIÓN DE NIVEL DE AUTOEFICACIA (Categórica)")
print("="*80)
print("\n", nivel_autoeficacia_data.value_counts().sort_index())
print("\nProporción (%):")
print(nivel_autoeficacia_data.value_counts(normalize=True).sort_index() * 100)

## 4. Análisis de Distribución y Normalidad

In [ ]:
print("\n" + "="*80)
print("ANÁLISIS DE DISTRIBUCIÓN - AUTOEFICACIA (0-10)")
print("="*80)

# Asimetría (Skewness)
skewness = skew(autoeficacia_data)
print(f"\nAsimetría (Skewness): {skewness:.4f}")
if abs(skewness) < 0.5:
    interpretacion_skew = "Distribución simétrica (aproximadamente normal)"
elif skewness > 0.5:
    interpretacion_skew = "Distribución sesgada a la derecha (positiva)"
else:
    interpretacion_skew = "Distribución sesgada a la izquierda (negativa)"
print(f"Interpretación: {interpretacion_skew}")

# Curtosis (Kurtosis)
kurt = kurtosis(autoeficacia_data)
print(f"\nCurtosis (Kurtosis): {kurt:.4f}")
if abs(kurt) < 0.5:
    interpretacion_kurt = "Curtosis normal (mesocúrt)ico)"
elif kurt > 0.5:
    interpretacion_kurt = "Distribución léptica (picos pronunciados)"
else:
    interpretacion_kurt = "Distribución platícurtica (más plana)"
print(f"Interpretación: {interpretacion_kurt}")

# Test de normalidad: Shapiro-Wilk
print(f"\n" + "-"*80)
print("Test de Normalidad: Shapiro-Wilk")
print("-"*80)
w_stat, p_value_sw = shapiro(autoeficacia_data)
print(f"Estadístico W:       {w_stat:.6f}")
print(f"P-valor:              {p_value_sw:.6f}")
print(f"Significancia (α=0.05):")
if p_value_sw < 0.05:
    print(f"  ⚠️  RECHAZAR H0: Datos NO distribuidos normalmente (p < 0.05)")
else:
    print(f"  ✓ NO RECHAZAR H0: Datos aproximadamente distribuidos normalmente (p ≥ 0.05)")

# Test de normalidad: Kolmogorov-Smirnov
print(f"\n" + "-"*80)
print("Test de Normalidad: Kolmogorov-Smirnov")
print("-"*80)
ks_stat, p_value_ks = kstest(autoeficacia_data, 'norm', args=(autoeficacia_data.mean(), autoeficacia_data.std()))
print(f"Estadístico D:       {ks_stat:.6f}")
print(f"P-valor:              {p_value_ks:.6f}")
print(f"Significancia (α=0.05):")
if p_value_ks < 0.05:
    print(f"  ⚠️  RECHAZAR H0: Datos NO distribuidos normalmente (p < 0.05)")
else:
    print(f"  ✓ NO RECHAZAR H0: Datos aproximadamente distribuidos normalmente (p ≥ 0.05)")

## 5. Visualizaciones Profesionales

In [ ]:
# Crear el directorio para guardar gráficos si no existe
import os
output_dir = '.'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("\n" + "="*80)
print("GENERANDO VISUALIZACIONES")
print("="*80)

# Gráfico 1: Histograma con Densidad KDE
print("\n1. Generando histogramas con densidad KDE...")
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histograma numérico
axes[0].hist(autoeficacia_data, bins=30, density=True, alpha=0.7, color='steelblue', edgecolor='black')
kde_x = np.linspace(autoeficacia_data.min(), autoeficacia_data.max(), 100)
kde = stats.gaussian_kde(autoeficacia_data)
axes[0].plot(kde_x, kde(kde_x), 'r-', linewidth=2.5, label='KDE')
axes[0].axvline(autoeficacia_data.mean(), color='green', linestyle='--', linewidth=2.5, label=f'Media: {autoeficacia_data.mean():.2f}')
axes[0].axvline(autoeficacia_data.median(), color='orange', linestyle='--', linewidth=2.5, label=f'Mediana: {autoeficacia_data.median():.2f}')
axes[0].set_xlabel('Autoeficacia (0-10)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Densidad', fontsize=12, fontweight='bold')
axes[0].set_title('Distribución de Autoeficacia - Escala Numérica', fontsize=13, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Histograma categórico
nivel_counts = nivel_autoeficacia_data.value_counts()
orden_niveles = ['Muy baja', 'Baja', 'Media', 'Alta', 'Muy alta']
nivel_counts_ordenado = nivel_counts.reindex([x for x in orden_niveles if x in nivel_counts.index])
colors = ['#d62728', '#ff7f0e', '#ffdd57', '#90ee90', '#2ca02c']
axes[1].bar(range(len(nivel_counts_ordenado)), nivel_counts_ordenado.values, color=colors[:len(nivel_counts_ordenado)], edgecolor='black', linewidth=1.5)
axes[1].set_xticks(range(len(nivel_counts_ordenado)))
axes[1].set_xticklabels(nivel_counts_ordenado.index, fontsize=11, rotation=45, ha='right')
axes[1].set_ylabel('Frecuencia', fontsize=12, fontweight='bold')
axes[1].set_title('Distribución de Nivel de Autoeficacia - Categórico', fontsize=13, fontweight='bold')
for i, v in enumerate(nivel_counts_ordenado.values):
    axes[1].text(i, v + 5, str(v), ha='center', fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('01_histogramas_densidad_autoeficacia.png', dpi=300, bbox_inches='tight')
print("   ✓ Guardado: 01_histogramas_densidad_autoeficacia.png")
plt.close()

# Gráfico 2: Box Plots
print("2. Generando box plots...")
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Box plot numérico
bp1 = axes[0].boxplot(autoeficacia_data, vert=True, patch_artist=True, widths=0.5)
for patch in bp1['boxes']:
    patch.set_facecolor('lightblue')
    patch.set_linewidth(2)
bp1['medians'][0].set_linewidth(2.5)
bp1['medians'][0].set_color('red')
axes[0].set_ylabel('Autoeficacia (0-10)', fontsize=12, fontweight='bold')
axes[0].set_title('Box Plot - Autoeficacia Numérica', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_xticklabels(['Autoeficacia'])

# Box plot por categoría
data_by_nivel = [panel_maestro[panel_maestro['nivel_autoeficacia'] == nivel]['autoeficacia_3_anos'].dropna().values 
                  for nivel in orden_niveles if nivel in panel_maestro['nivel_autoeficacia'].values]
bp2 = axes[1].boxplot(data_by_nivel, vert=True, patch_artist=True, labels=[x for x in orden_niveles if x in panel_maestro['nivel_autoeficacia'].values], widths=0.6)
for patch in bp2['boxes']:
    patch.set_facecolor('lightgreen')
    patch.set_linewidth(2)
bp2['medians'][0].set_linewidth(2.5)
bp2['medians'][0].set_color('red')
axes[1].set_ylabel('Autoeficacia (0-10)', fontsize=12, fontweight='bold')
axes[1].set_title('Box Plot - Autoeficacia por Nivel', fontsize=13, fontweight='bold')
axes[1].set_xticklabels([x for x in orden_niveles if x in panel_maestro['nivel_autoeficacia'].values], rotation=45, ha='right')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('02_boxplots_autoeficacia.png', dpi=300, bbox_inches='tight')
print("   ✓ Guardado: 02_boxplots_autoeficacia.png")
plt.close()

# Gráfico 3: Violin Plots
print("3. Generando violin plots...")
fig, ax = plt.subplots(figsize=(14, 7))

data_for_violin = []
labels_for_violin = []
for nivel in orden_niveles:
    if nivel in panel_maestro['nivel_autoeficacia'].values:
        data_for_violin.append(panel_maestro[panel_maestro['nivel_autoeficacia'] == nivel]['autoeficacia_3_anos'].dropna().values)
        labels_for_violin.append(nivel)

parts = ax.violinplot(data_for_violin, positions=range(len(data_for_violin)), showmeans=True, showmedians=True)
ax.set_xticks(range(len(labels_for_violin)))
ax.set_xticklabels(labels_for_violin, fontsize=12, rotation=45, ha='right')
ax.set_ylabel('Autoeficacia (0-10)', fontsize=12, fontweight='bold')
ax.set_title('Violin Plot - Distribución de Autoeficacia por Nivel', fontsize=13, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('03_violinplots_autoeficacia.png', dpi=300, bbox_inches='tight')
print("   ✓ Guardado: 03_violinplots_autoeficacia.png")
plt.close()

# Gráfico 4: Q-Q Plots
print("4. Generando Q-Q plots...")
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Q-Q plot para toda la muestra
stats.probplot(autoeficacia_data, dist="norm", plot=axes[0])
axes[0].set_title('Q-Q Plot - Autoeficacia (Todas las Observaciones)', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Q-Q plot normalizado
autoeficacia_normalizada = (autoeficacia_data - autoeficacia_data.mean()) / autoeficacia_data.std()
stats.probplot(autoeficacia_normalizada, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot - Autoeficacia Normalizada', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('04_qqplots_autoeficacia.png', dpi=300, bbox_inches='tight')
print("   ✓ Guardado: 04_qqplots_autoeficacia.png")
plt.close()

# Gráfico 5: Distribución por Nivel
print("5. Generando gráfico de distribución por nivel...")
fig, ax = plt.subplots(figsize=(14, 8))

nivel_counts_prop = (nivel_autoeficacia_data.value_counts() / len(nivel_autoeficacia_data) * 100)
nivel_counts_prop = nivel_counts_prop.reindex([x for x in orden_niveles if x in nivel_counts_prop.index])

colors_pie = ['#d62728', '#ff7f0e', '#ffdd57', '#90ee90', '#2ca02c']
wedges, texts, autotexts = ax.pie(nivel_counts_prop.values, labels=nivel_counts_prop.index, autopct='%1.1f%%',
                                     colors=colors_pie[:len(nivel_counts_prop)], startangle=90, textprops={'fontsize': 12, 'fontweight': 'bold'})
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontsize(11)
    autotext.set_fontweight('bold')

ax.set_title('Proporción de Estudiantes por Nivel de Autoeficacia', fontsize=13, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('05_distribucion_nivel_autoeficacia.png', dpi=300, bbox_inches='tight')
print("   ✓ Guardado: 05_distribucion_nivel_autoeficacia.png")
plt.close()

# Gráfico 6: Comparación categórica
print("6. Generando gráfico comparativo de autoeficacia...")
fig, ax = plt.subplots(figsize=(14, 8))

nivel_stats = {}
for nivel in orden_niveles:
    if nivel in panel_maestro['nivel_autoeficacia'].values:
        datos_nivel = panel_maestro[panel_maestro['nivel_autoeficacia'] == nivel]['autoeficacia_3_anos'].dropna()
        nivel_stats[nivel] = {
            'mean': datos_nivel.mean(),
            'std': datos_nivel.std(),
            'count': len(datos_nivel)
        }

x_pos = np.arange(len(nivel_stats))
means = [nivel_stats[nivel]['mean'] for nivel in nivel_stats.keys()]
stds = [nivel_stats[nivel]['std'] for nivel in nivel_stats.keys()]

bars = ax.bar(x_pos, means, yerr=stds, capsize=10, color=colors_pie[:len(nivel_stats)], 
               edgecolor='black', linewidth=2, alpha=0.8, error_kw={'linewidth': 2, 'ecolor': 'gray'})

ax.set_xticks(x_pos)
ax.set_xticklabels(nivel_stats.keys(), fontsize=12, rotation=45, ha='right', fontweight='bold')
ax.set_ylabel('Autoeficacia Media (0-10)', fontsize=12, fontweight='bold')
ax.set_title('Media de Autoeficacia por Nivel (con Desviación Estándar)', fontsize=13, fontweight='bold')
ax.set_ylim(0, 10)
ax.grid(True, alpha=0.3, axis='y')

for i, (mean, std, nivel) in enumerate(zip(means, stds, nivel_stats.keys())):
    count = nivel_stats[nivel]['count']
    ax.text(i, mean + std + 0.3, f'{mean:.2f}\n(n={count})', ha='center', fontweight='bold', fontsize=10)

plt.tight_layout()
plt.savefig('06_autoeficacia_por_categoria.png', dpi=300, bbox_inches='tight')
print("   ✓ Guardado: 06_autoeficacia_por_categoria.png")
plt.close()

print("\n✓ Todas las visualizaciones generadas exitosamente")

## 6. Detección de Outliers

In [ ]:
print("\n" + "="*80)
print("DETECCIÓN DE OUTLIERS")
print("="*80)

# Método 1: IQR (Rango Intercuartílico)
print(f"\nMÉTODO 1: IQR (Rango Intercuartílico)")
print("-" * 80)
q1 = autoeficacia_data.quantile(0.25)
q3 = autoeficacia_data.quantile(0.75)
iqr = q3 - q1
lower_bound_iqr = q1 - 1.5 * iqr
upper_bound_iqr = q3 + 1.5 * iqr

outliers_iqr = autoeficacia_data[(autoeficacia_data < lower_bound_iqr) | (autoeficacia_data > upper_bound_iqr)]
print(f"Límite inferior: {lower_bound_iqr:.4f}")
print(f"Límite superior: {upper_bound_iqr:.4f}")
print(f"Número de outliers: {len(outliers_iqr)} ({len(outliers_iqr)/len(autoeficacia_data)*100:.2f}%)")
if len(outliers_iqr) > 0:
    print(f"Valores outliers: {sorted(outliers_iqr.values)[:10]}") # Mostrar primeros 10

# Método 2: Z-score (|Z| > 3)
print(f"\nMÉTODO 2: Z-score (|Z| > 3)")
print("-" * 80)
z_scores = np.abs(stats.zscore(autoeficacia_data))
outliers_zscore = autoeficacia_data[z_scores > 3]
print(f"Número de outliers: {len(outliers_zscore)} ({len(outliers_zscore)/len(autoeficacia_data)*100:.2f}%)")
if len(outliers_zscore) > 0:
    print(f"Valores outliers: {sorted(outliers_zscore.values)[:10]}") # Mostrar primeros 10

# Método 3: Isolation Forest
print(f"\nMÉTODO 3: Isolation Forest")
print("-" * 80)
iso_forest = IsolationForest(contamination=0.1, random_state=42)
outliers_if = iso_forest.fit_predict(autoeficacia_data.values.reshape(-1, 1))
n_outliers_if = (outliers_if == -1).sum()
print(f"Número de outliers: {n_outliers_if} ({n_outliers_if/len(autoeficacia_data)*100:.2f}%)")

# Resumen
print(f"\n" + "="*80)
print("RESUMEN DE OUTLIERS")
print("="*80)
print(f"Método IQR:          {len(outliers_iqr)} outliers ({len(outliers_iqr)/len(autoeficacia_data)*100:.2f}%)")
print(f"Método Z-score:      {len(outliers_zscore)} outliers ({len(outliers_zscore)/len(autoeficacia_data)*100:.2f}%)")
print(f"Isolation Forest:    {n_outliers_if} outliers ({n_outliers_if/len(autoeficacia_data)*100:.2f}%)")
print(f"\n💡 Se recomienda usar el método IQR para identificar outliers.")

## 7. Evaluación de Calidad de Datos

In [ ]:
print("\n" + "="*80)
print("EVALUACIÓN DE CALIDAD DE DATOS")
print("="*80)

# Datos faltantes
print(f"\nDATA FALTANTES:")
print("-" * 80)
for col in autoeficacia_cols:
    missing_count = panel_maestro[col].isna().sum()
    missing_pct = (missing_count / len(panel_maestro)) * 100
    print(f"{col}:")
    print(f"  Total de observaciones: {len(panel_maestro)}")
    print(f"  Datos faltantes: {missing_count} ({missing_pct:.2f}%)")
    print(f"  Datos válidos: {len(panel_maestro) - missing_count} ({100-missing_pct:.2f}%)")

# Duplicados
print(f"\nDUPLICADOS:")
print("-" * 80)
duplicados_total = panel_maestro[autoeficacia_cols].duplicated().sum()
print(f"Número de filas duplicadas: {duplicados_total}")

# Valores únicos
print(f"\nVALORES ÚNICOS:")
print("-" * 80)
for col in autoeficacia_cols:
    unique_count = panel_maestro[col].nunique()
    print(f"{col}: {unique_count} valores únicos")

# Consistencia de datos
print(f"\nCONSISTENCIA DE DATOS:")
print("-" * 80)
print(f"Autoeficacia numérica (rango esperado: 0-10):")
print(f"  Mínimo: {autoeficacia_data.min():.4f} {'✓' if autoeficacia_data.min() >= 0 else '❌'}")
print(f"  Máximo: {autoeficacia_data.max():.4f} {'✓' if autoeficacia_data.max() <= 10 else '❌'}")

print(f"\nNivel de autoeficacia (categorías esperadas):")
categorias_esperadas = set(['Muy baja', 'Baja', 'Media', 'Alta', 'Muy alta'])
categorias_observadas = set(nivel_autoeficacia_data.unique())
print(f"  Categorías esperadas: {categorias_esperadas}")
print(f"  Categorías observadas: {categorias_observadas}")
print(f"  ✓ Consistente" if categorias_observadas.issubset(categorias_esperadas) else "  ❌ Inconsistente")

# Correlación entre variables
print(f"\nRELACIÓN ENTRE VARIABLES NUMÉRICAS Y CATEGÓRICAS:")
print("-" * 80)
print(f"Coeficiente de correlación entre autoeficacia numérica y nivel:")
# Convertir nivel a numérico para correlación
nivel_numeric_map = {'Muy baja': 1, 'Baja': 2, 'Media': 3, 'Alta': 4, 'Muy alta': 5}
panel_maestro['nivel_numeric'] = panel_maestro['nivel_autoeficacia'].map(nivel_numeric_map)
correlacion = panel_maestro['autoeficacia_3_anos'].corr(panel_maestro['nivel_numeric'])
print(f"  Pearson r: {correlacion:.4f}")
print(f"  {'✓ Fuerte correlación positiva esperada' if correlacion > 0.8 else '⚠️ Correlación moderada/débil - revisar consistencia'}")

print("\n✓ Evaluación de calidad completada")

## 8. Resumen Ejecutivo

In [ ]:
print("\n" + "="*80)
print("RESUMEN EJECUTIVO - ANÁLISIS DE AUTOEFICACIA")
print("="*80)

print(f"\n📊 ESTADÍSTICAS PRINCIPALES:")
print(f"  • Media de autoeficacia: {autoeficacia_data.mean():.2f}/10")
print(f"  • Mediana de autoeficacia: {autoeficacia_data.median():.2f}/10")
print(f"  • Desviación estándar: {autoeficacia_data.std():.2f}")
print(f"  • Rango: {autoeficacia_data.min():.2f} - {autoeficacia_data.max():.2f}")

print(f"\n📈 DISTRIBUCIÓN:")
if abs(skewness) < 0.5:
    dist_desc = "aproximadamente simétrica"
elif skewness > 0.5:
    dist_desc = "sesgada hacia la derecha (más estudiantes con autoeficacia alta)"
else:
    dist_desc = "sesgada hacia la izquierda (más estudiantes con autoeficacia baja)"

print(f"  • Forma: {dist_desc}")
print(f"  • Asimetría (Skewness): {skewness:.4f}")
print(f"  • Curtosis (Kurtosis): {kurt:.4f}")

print(f"\n🔬 NORMALIDAD:")
print(f"  • Shapiro-Wilk p-valor: {p_value_sw:.6f}")
print(f"  • Kolmogorov-Smirnov p-valor: {p_value_ks:.6f}")
print(f"  • Conclusión: Datos {'NO ' if p_value_sw < 0.05 else ''}distribuidos normalmente")

print(f"\n⚠️ OUTLIERS (Método IQR):")
print(f"  • Número de outliers: {len(outliers_iqr)} ({len(outliers_iqr)/len(autoeficacia_data)*100:.2f}%)")
print(f"  • Límites: [{lower_bound_iqr:.2f}, {upper_bound_iqr:.2f}]")

print(f"\n🎯 DISTRIBUCIÓN POR NIVEL:")
for nivel in orden_niveles:
    if nivel in nivel_autoeficacia_data.value_counts().index:
        count = (nivel_autoeficacia_data == nivel).sum()
        pct = count / len(nivel_autoeficacia_data) * 100
        print(f"  • {nivel}: {count} estudiantes ({pct:.1f}%)")

print(f"\n📋 CALIDAD DE DATOS:")
print(f"  • Registros válidos: {len(autoeficacia_data)}/{len(panel_maestro)} ({len(autoeficacia_data)/len(panel_maestro)*100:.1f}%)")
print(f"  • Datos faltantes: {panel_maestro['autoeficacia_3_anos'].isna().sum()} ({panel_maestro['autoeficacia_3_anos'].isna().sum()/len(panel_maestro)*100:.1f}%)")
print(f"  • Duplicados: {panel_maestro[autoeficacia_cols].duplicated().sum()}")
print(f"  • Estado: {'✓ Buena calidad' if len(autoeficacia_data)/len(panel_maestro) > 0.80 else '⚠️ Calidad moderada' if len(autoeficacia_data)/len(panel_maestro) > 0.60 else '❌ Calidad baja'}")

print(f"\n✅ CONCLUSIONES:")
if autoeficacia_data.mean() >= 6:
    conclusión_media = "La autoeficacia promedio es ALTA"
elif autoeficacia_data.mean() >= 4:
    conclusión_media = "La autoeficacia promedio es MEDIA"
else:
    conclusión_media = "La autoeficacia promedio es BAJA"

print(f"  • {conclusión_media}")
print(f"  • Los datos {'muestran una distribución normal' if p_value_sw >= 0.05 else 'NO muestran una distribución normal'}")
print(f"  • Hay {'pocos' if len(outliers_iqr)/len(autoeficacia_data) < 0.05 else 'varios'} outliers que podrían afectar el análisis")
print(f"  • La calidad de datos es {'aceptable' if len(autoeficacia_data)/len(panel_maestro) > 0.80 else 'moderada - revisar datos faltantes'}")

print(f"\n" + "="*80)
print("FIN DEL ANÁLISIS")
print("="*80)